# Analyzing LAPD's 'Part I' crimes database: 2010-present

### Import data tools

In [1]:
import pandas as pd
import geopandas as gpd
import pyarrow
import matplotlib
import matplotlib.pyplot as plt
import geojson
import json
import jenkspy
import numpy as np
from earthpy import clip as cl
from altair import datum
import weightedcalcs as wc
import altair as alt
alt.renderers.enable('notebook')
import altair_latimes as lat
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1000

### Read crimes CSV downloaded from LA City data portal

In [2]:
# data exported from '00-lapd-crimes-processing.ipynb'
crimes = pd.read_feather('/Users/mhustiles/data/data/LA/crimes.feather')

---

### Understanding LAPD's modus operandi codes

In [3]:
mocrimes = crimes.dropna(subset=['modus_operandi_code'])

In [4]:
mocodes = pd.read_csv('mo_codes.csv')
mocodes.head()

mo_code  mo_code_description
0    0100  Suspect Impersonate
1    0101           Aid victim
2    0102                Blind
3    0103             Crippled
4    0104             Customer

### Isolating cases involving specific 'MO' tags (ie homeless, , gang, etc)

In [5]:
# For example...
mocodes_homeless = mocodes[mocodes['mo_code_description'].str.lower().str.contains('gang')]
mocodes_homeless.head()

mo_code                                mo_code_description
114    0371  Gang affiliation questions asked/made gang sta...
117    0374            Gang signs/threw gang signs using hands
270    0906                                              Gangs
312    0946                                          Gang Feud
410    1270                             Victim was gang member

### Other filters

In [6]:
# Specific street
rubens = crimes[crimes['address'].str.contains('RUBENS', na=False)]
# Crimes tagged by specific modus operandi code
bunco = crimes[crimes['modus_operandi_code'].str.contains('0800', na=False) ]

---

## Basic aggregates on variables for all crimes

### Which crimes are most common? 

In [7]:
violations = crimes.groupby(['crime_code_description'])\
.agg('size').reset_index(name='total').sort_values(by='total', ascending=False)

In [8]:
violations.head(10)

crime_code_description   total
9                               Burglary from vehicle  162050
54                                   Vehicle - stolen  159722
42                 Theft plain - petty ($950 & under)  149561
8                                            Burglary  147409
2      Assault with deadly weapon, aggravated assault   92632
39    Theft from motor vehicle - petty ($950 & under)   88368
29                                            Robbery   83813
47  Theft-grand ($950.01 & over)excpt,guns,fowl,li...   74299
32           Shoplifting - petty theft ($950 & under)   48380
38   Theft from motor vehicle - grand ($400 and over)   31605

### Places where crimes occur?

In [9]:
places = crimes.groupby(['premises_description'])\
.agg('size').reset_index(name='total').sort_values(by='total', ascending=False)

In [10]:
places.head(10)

premises_description   total
273                                        Street  358571
262                        Single family dwelling  146704
229                                   Parking lot  115353
207  Multi-unit dwelling (apartment, duplex, etc)   85681
261                                      Sidewalk   61525
220                                Other business   50238
64                                       Driveway   34295
80                                 Garage/carport   33409
56                               Department store   26478
108                                        Market   18316

### Areas of the city? 

In [11]:
divisions = crimes.groupby(['division_name'])\
.agg('size').reset_index(name='total').sort_values(by='total', ascending=False)

In [12]:
divisions.head(10)

division_name  total
0    77th Street  79948
15     Southwest  75950
12       Pacific  67790
8    N Hollywood  63241
14     Southeast  61532
9         Newton  57847
10     Northeast  55868
6      Hollywood  55114
1        Central  55076
2     Devonshire  54583

### What kinds of weapons?

In [13]:
weapons = crimes.groupby(['weapon_description'])\
.agg('size').reset_index(name='total').sort_values(by='total', ascending=False)

In [14]:
weapons.head(10)

weapon_description   total
65  Strong-arm (hands, fist, feet or bodily force)  105348
75                     Unknown weapon/other weapon   33139
27                                        Hand gun   31527
58                           Semi-automatic pistol   12050
32                Knife with blade 6inches or less   11801
44                                     Other knife    8993
77                                         Vehicle    6615
73                                 Unknown firearm    6265
50                                        Revolver    5203
78                                   Verbal threat    5121

### Which days of the week?

In [15]:
weekdays = crimes.groupby(['weekday'])\
.agg('size').reset_index(name='total').sort_values(by='total', ascending=False)

In [16]:
weekdays.head(7)

weekday   total
0     Friday  181811
2   Saturday  170304
6  Wednesday  165761
4   Thursday  165681
1     Monday  163879
5    Tuesday  163442
3     Sunday  159098

---